In [93]:
import pandas as pd 
import numpy as np


In [94]:
df=pd.read_csv("spotify_millsongdata.csv")
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [95]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [96]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [97]:
df['text'][0]

"Death is the best  \r\nBetter than all the rest  \r\nOn the dutiful lies in the rock paradise  \r\nLosses and wear and the texture of age  \r\nAdds a truth to the heart  \r\nAnd a light to the face  \r\n  \r\nIt's good when you don't give as  \r\nMuch of a fuck  \r\nAs you did when the threats mount  \r\nTo make you suck  \r\nA burst of rage  \r\nA lively cry  \r\nEmotions wide  \r\n  \r\nYour death is certain for sure  \r\nMy death is certain alright  \r\nBut death is certain no cure  \r\n'Cause death is certain  \r\n'Cause death is certain so why?  \r\n  \r\nGotta die  \r\nA bone and a skull  \r\nIn a conjugal hell  \r\nA mall full of gall  \r\nAnd the music jerks off  \r\nPale and fate  \r\n  \r\nAt the table they sat  \r\nWith their necks choking tight  \r\nIn their collars dead white  \r\nLiquor and food 'till  \r\nThey spin and explode  \r\nIn a shower of crap  \r\nIn America's lap  \r\nA pliceman's numb  \r\nA lazy bum  \r\nEmotions come  \r\n  \r\n'Cause death is certain it's 

In [98]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ',regex =True)

In [99]:
df.tail(5)

,artist,song,text
4995,Pharrell Williams,Mamacita,"mamacita, oh my god i think you're on fire \r..."
4996,Britney Spears,Don't Let Me Be The Last To Know,my friends say you're so into me \r and that ...
4997,Bob Marley,Is This Love,"i want to love you, and treat you right, \r i..."
4998,Selena Gomez,I Promise You,i know \r that my love for \r you is real \...
4999,James Taylor,Angry Blues,i can't help it if i don't feel so good. \r i...


In [76]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [100]:
import nltk
from nltk.stem.porter import PorterStemmer

In [101]:
stemmer=PorterStemmer()

In [102]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [103]:
token("you are beutiful")

'you are beuti'

In [104]:
df['text'].apply(lambda x :token(x))

0       death is the best better than all the rest on ...
1       i 've alway been crazi and the troubl that it ...
2       i saw two lover walk along the sandi shore her...
3       oh , brother , man where are you ? i am buckli...
4       out of my sight but the vision is still the sa...
                              ...                        
4995    mamacita , oh my god i think you 're on fire d...
4996    my friend say you 're so into me and that you ...
4997    i want to love you , and treat you right , i w...
4998    i know that my love for you is real it 's some...
4999    i ca n't help it if i do n't feel so good . i ...
Name: text, Length: 5000, dtype: object

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [106]:
tfid=TfidfVectorizer(analyzer='word', stop_words='english')

In [107]:
matrix=tfid.fit_transform(df['text'])

In [108]:
similer=cosine_similarity(matrix)

In [109]:
similer[0]

array([1.        , 0.01590085, 0.08116188, ..., 0.00462747, 0.0421706 ,
       0.01332226])

In [111]:
df[df['song']=='Mamacita'].index[0]

4995

In [115]:
def recommender(song_name):
    idx= df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similer[idx])), reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:21]:
        song.append(df.iloc[s_id[0]].song)
    return song    

In [116]:
recommender("Mamacita")

['When I Think About Leaving',
 'Little Black Dress',
 'I Wanna Grow Old With You',
 'Throw This Away',
 'I Wanna Believe You',
 'Move On',
 'I Want You',
 'I Just Wanna Be There',
 'Tell All The World About You',
 'More',
 'Wanna Be There',
 "I Think I'm In Love",
 'Be On You',
 'Without You',
 'Dance With Me',
 "I Don't Think We Could Ever Be Friends",
 'Love Me No More',
 "It's Over",
 'Downpour',
 'Will You Marry Me']

In [117]:
import pickle

In [119]:
pickle.dump(similer,open("similarity","wb"))

In [120]:
pickle.dump(df,open("df","wb"))